In [ ]:
import pandas as pd

In [ ]:
!pip install folium

# General Map

In [ ]:
import requests
import folium

m = folium.Map([43, -100], zoom_start=4)

us_states = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
).json()

folium.Choropleth(
    geo_data=us_states,
    fill_opacity=0.3,
    line_weight=2,
).add_to(m)

m

# General Maps Plus Circular Markers

In [ ]:
import folium
import requests

# Initialize the map centered on the US (from snippet 2)
# Note: You might want to adjust the zoom or location if you want both areas visible initially.
m = folium.Map(location=[43, -100], zoom_start=4)

# --- Brazil Markers (from Snippet 1) ---
radius = 50
folium.CircleMarker(
    location=[-27.55, -48.8],
    radius=radius,
    color="cornflowerblue",
    stroke=False,
    fill=True,
    fill_opacity=0.6,
    opacity=1,
    popup="{} pixels".format(radius),
    tooltip="I am in pixels",
).add_to(m)

radius = 25
folium.CircleMarker(
    location=[-27.35, -48.8],
    radius=radius,
    color="black",
    weight=3,
    fill=False,
    fill_opacity=0.6,
    opacity=1,
).add_to(m)

radius = 10000
folium.Circle(
    location=[-27.551667, -48.478889],
    radius=radius,
    color="black",
    weight=1,
    fill_opacity=0.6,
    opacity=1,
    fill_color="green",
    fill=False,  # gets overridden by fill_color
    popup="{} meters".format(radius),
    tooltip="I am in meters",
).add_to(m)

# --- US Choropleth (from Snippet 2) ---
# Load GeoJSON data
us_states = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
).json()

# Add Choropleth layer
folium.Choropleth(
    geo_data=us_states,
    fill_opacity=0.3,
    line_weight=2,
).add_to(m)

# Return the map object
m

# Build Counties Map

In [ ]:
import folium
import requests
import json
from folium import plugins
from folium.plugins import FloatImage
from jinja2 import Template

# ---------------------------------------------------------
# 1. DATA PREPARATION
# ---------------------------------------------------------

# A. Fetch the US Counties TopoJSON
topo_url = "https://cdn.jsdelivr.net/npm/us-atlas@3/counties-10m.json"
us_counties_topo = requests.get(topo_url).json()

# B. Define the Icons (URLs from your HTML)
icon_urls = {
    "Fixed Surface Buoy": "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/buoy-gray.png",
    "Fixed Coastal Station": "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/shore_station-gray.png",
    "HFRadar Site": "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/hfradar-gray.png",
    "Fixed Bottom Station": "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/circ-gray.png",
    "Fixed Bottom Mount Mooring": "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/tri-gray.png"
}

# C. Define Status Colors (Matches your HTML CSS)
status_colors = {
    "Existing": "#6bce70",
    "Permitting/Construction": "#fdf341",
    "Proposed": "#fc7b42"
}

# D. Generate Mock Data (To demonstrate the labeling)
# When you have your actual data, replace this list with your own.
mock_points = [
    {"lat": 32.0, "lon": -80.0, "type": "Fixed Surface Buoy", "status": "Existing", "label": "Savannah Buoy"},
    {"lat": 33.5, "lon": -78.5, "type": "Fixed Coastal Station", "status": "Proposed", "label": "Myrtle Beach Station"},
    {"lat": 29.5, "lon": -81.0, "type": "HFRadar Site", "status": "Existing", "label": "Daytona Radar"},
    {"lat": 34.0, "lon": -77.0, "type": "Fixed Bottom Station", "status": "Permitting/Construction", "label": "Wilmington Deep"},
    {"lat": 28.0, "lon": -80.0, "type": "Fixed Bottom Mount Mooring", "status": "Existing", "label": "Cape Canaveral Mooring"},
]

# ---------------------------------------------------------
# 2. VISUALIZATION
# ---------------------------------------------------------

# Initialize the map with a Dark theme to match your snippet
m = folium.Map(location=[31.5, -80], zoom_start=6, tiles="CartoDB dark_matter")

# A. Add the TopoJSON Counties Layer
# We style it with thin, subtle lines to look "beautiful" and professional
folium.TopoJson(
    data=us_counties_topo,
    object_path='objects.counties',  # Key inside the TopoJSON file
    name='US Counties',
    style_function=lambda x: {
        'color': '#444444',    # Dark grey border
        'weight': 0.5,         # Very thin line
        'fillColor': '#000000',
        'fillOpacity': 0.0     # Transparent fill so the dark map shows through
    }
).add_to(m)

# B. Add the Points with Custom Icons
for point in mock_points:
    # 1. Create the Custom Icon
    icon = folium.CustomIcon(
        icon_image=icon_urls[point["type"]],
        icon_size=(20, 20),  # Adjust size as needed
        icon_anchor=(10, 10)
    )

    # 2. Create the Popup HTML (The "Label")
    # We style the status text using the colors from your legend
    popup_html = f"""
    <div style="font-family: sans-serif; width: 150px;">
        <b>{point['label']}</b><br>
        Type: {point['type']}<br>
        Status: <span style="color:{status_colors[point['status']]}; font-weight:bold;">{point['status']}</span>
    </div>
    """

    # 3. Add Marker to Map
    folium.Marker(
        location=[point["lat"], point["lon"]],
        icon=icon,
        popup=folium.Popup(popup_html, max_width=200),
        tooltip=point['label'] # Hover text
    ).add_to(m)

# C. Add the "Rose" Image (Top Right)
rose_url = "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/rose.png"
FloatImage(rose_url, bottom=85, left=85).add_to(m) # Positioning in Folium is percentage based (0-100)


# D. Inject the Custom HTML Legend
# We adapt your CSS to work within Folium's floating element
legend_html = """
{% macro html(this, kwargs) %}
<style>
.map-legend {
    position: absolute;
    bottom: 50px;
    left: 10px;
    width: 200px;
    z-index: 9999;
    background-color: rgba(255, 255, 255, 0.8);
    padding: 10px;
    border: 1px solid #bbb;
    border-radius: 5px;
    font-family: sans-serif;
    font-size: 12px;
}
.legend-title {
    font-weight: bold;
    margin-bottom: 5px;
    display: block;
}
.legend-list {
    list-style: none;
    margin: 0;
    padding: 0;
    margin-bottom: 10px;
}
.legend-item {
    display: flex;
    align-items: center;
    margin-bottom: 4px;
}
.legend-color {
    width: 15px;
    height: 15px;
    margin-right: 8px;
    border: 1px solid #bbb;
    display: inline-block;
}
.legend-icon {
    width: 15px;
    height: 15px;
    margin-right: 8px;
}
</style>

<div class="map-legend">
    <span class="legend-title">Asset Status</span>
    <ul class="legend-list">
        <li class="legend-item"><span class="legend-color" style="background:#6bce70;"></span>Existing</li>
        <li class="legend-item"><span class="legend-color" style="background:#fdf341;"></span>Permitting</li>
        <li class="legend-item"><span class="legend-color" style="background:#fc7b42;"></span>Proposed</li>
    </ul>

    <span class="legend-title">Asset Type</span>
    <ul class="legend-list">
        <li class="legend-item"><img class="legend-icon" src="https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/buoy-gray.png">Fixed Surface Buoy</li>
        <li class="legend-item"><img class="legend-icon" src="https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/shore_station-gray.png">Fixed Coastal Station</li>
        <li class="legend-item"><img class="legend-icon" src="https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/hfradar-gray.png">HFRadar Site</li>
        <li class="legend-item"><img class="legend-icon" src="https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/circ-gray.png">Fixed Bottom Station</li>
        <li class="legend-item"><img class="legend-icon" src="https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/tri-gray.png">Bottom Mount Mooring</li>
    </ul>
</div>
{% endmacro %}
"""

macro = folium.MacroElement()
macro._template = Template(legend_html)
m.get_root().add_child(macro)

# Return map
m

# counties map colored

In [ ]:
import folium
import requests
import json
import random
from folium import plugins
from folium.plugins import FloatImage
from jinja2 import Template

# ---------------------------------------------------------
# 1. DATA PREPARATION
# ---------------------------------------------------------

# Fetch the US Counties TopoJSON
topo_url = "https://cdn.jsdelivr.net/npm/us-atlas@3/counties-10m.json"
us_counties_topo = requests.get(topo_url).json()

# --- HELPER: Random Color Generator ---
# We will use this to color each county differently
def random_color(feature):
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Define Icons and Colors (Same as before)
icon_urls = {
    "Fixed Surface Buoy": "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/buoy-gray.png",
    "Fixed Coastal Station": "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/shore_station-gray.png",
    "HFRadar Site": "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/hfradar-gray.png",
    "Fixed Bottom Station": "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/circ-gray.png",
    "Fixed Bottom Mount Mooring": "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/tri-gray.png"
}

status_colors = {
    "Existing": "#6bce70",
    "Permitting/Construction": "#fdf341",
    "Proposed": "#fc7b42"
}

mock_points = [
    {"lat": 32.0, "lon": -80.0, "type": "Fixed Surface Buoy", "status": "Existing", "label": "Savannah Buoy"},
    {"lat": 33.5, "lon": -78.5, "type": "Fixed Coastal Station", "status": "Proposed", "label": "Myrtle Beach Station"},
    {"lat": 29.5, "lon": -81.0, "type": "HFRadar Site", "status": "Existing", "label": "Daytona Radar"},
    {"lat": 34.0, "lon": -77.0, "type": "Fixed Bottom Station", "status": "Permitting/Construction", "label": "Wilmington Deep"},
    {"lat": 28.0, "lon": -80.0, "type": "Fixed Bottom Mount Mooring", "status": "Existing", "label": "Cape Canaveral Mooring"},
]

# ---------------------------------------------------------
# 2. VISUALIZATION
# ---------------------------------------------------------

m = folium.Map(location=[31.5, -80], zoom_start=6, tiles="CartoDB dark_matter")

# --- CHANGED SECTION: Colored Counties ---
# We use 'style_function' to return a different fillColor for every feature (county)
folium.TopoJson(
    data=us_counties_topo,
    object_path='objects.counties',
    name='US Counties',
    style_function=lambda feature: {
        'color': '#222222',        # Darker border color
        'weight': 0.5,             # Thin border
        'fillColor': random_color(feature), # RANDOM COLOR!
        'fillOpacity': 0.3         # 30% opacity so we can still see the map labels underneath
    }
).add_to(m)

# Add Points
for point in mock_points:
    icon = folium.CustomIcon(
        icon_image=icon_urls[point["type"]],
        icon_size=(20, 20),
        icon_anchor=(10, 10)
    )
    popup_html = f"""
    <div style="font-family: sans-serif; width: 150px;">
        <b>{point['label']}</b><br>
        Type: {point['type']}<br>
        Status: <span style="color:{status_colors[point['status']]}; font-weight:bold;">{point['status']}</span>
    </div>
    """
    folium.Marker(
        location=[point["lat"], point["lon"]],
        icon=icon,
        popup=folium.Popup(popup_html, max_width=200),
        tooltip=point['label']
    ).add_to(m)

# Add Rose Image
rose_url = "https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/rose.png"
FloatImage(rose_url, bottom=85, left=85).add_to(m)

# Add Legend
legend_html = """
{% macro html(this, kwargs) %}
<style>
.map-legend {
    position: absolute;
    bottom: 50px;
    left: 10px;
    width: 200px;
    z-index: 9999;
    background-color: rgba(255, 255, 255, 0.8);
    padding: 10px;
    border: 1px solid #bbb;
    border-radius: 5px;
    font-family: sans-serif;
    font-size: 12px;
}
.legend-title { font-weight: bold; margin-bottom: 5px; display: block; }
.legend-list { list-style: none; margin: 0; padding: 0; margin-bottom: 10px; }
.legend-item { display: flex; align-items: center; margin-bottom: 4px; }
.legend-color { width: 15px; height: 15px; margin-right: 8px; border: 1px solid #bbb; display: inline-block; }
.legend-icon { width: 15px; height: 15px; margin-right: 8px; }
</style>

<div class="map-legend">
    <span class="legend-title">Asset Status</span>
    <ul class="legend-list">
        <li class="legend-item"><span class="legend-color" style="background:#6bce70;"></span>Existing</li>
        <li class="legend-item"><span class="legend-color" style="background:#fdf341;"></span>Permitting</li>
        <li class="legend-item"><span class="legend-color" style="background:#fc7b42;"></span>Proposed</li>
    </ul>
    <span class="legend-title">Asset Type</span>
    <ul class="legend-list">
        <li class="legend-item"><img class="legend-icon" src="https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/buoy-gray.png">Fixed Surface Buoy</li>
        <li class="legend-item"><img class="legend-icon" src="https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/shore_station-gray.png">Fixed Coastal Station</li>
        <li class="legend-item"><img class="legend-icon" src="https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/hfradar-gray.png">HFRadar Site</li>
        <li class="legend-item"><img class="legend-icon" src="https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/circ-gray.png">Fixed Bottom Station</li>
        <li class="legend-item"><img class="legend-icon" src="https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/tri-gray.png">Bottom Mount Mooring</li>
    </ul>
</div>
{% endmacro %}
"""
macro = folium.MacroElement()
macro._template = Template(legend_html)
m.get_root().add_child(macro)

m

# Evaluating MSA and ZCTA data so that we know what columns to work with


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/msa_spatial_2013_v4.csv")

In [ ]:
df.shape

(519, 29)

In [ ]:
df.columns

Index(['CBSA_CODE', 'DCI_YEAR', 'n_zctas_in_msa', 'calc_msa_population',
       'year_globalMorans_I', 'global_morans_p', 'spatial_gini_dci',
       'mean_dci_weighted', 'pct_pop_in_hh_clusters', 'pct_pop_in_ll_clusters',
       'pct_pop_hl_outliers', 'pct_pop_lh_outliers', 'pct_pop_in_distressed',
       'pct_pop_in_prosperous', 'pct_pop_at_risk_or_distressed',
       'pct_pop_in_distressed_hotspots', 'pct_pop_in_prosperous_coldspots',
       'pct_pop_in_distressed_coldspots', 'pct_pop_in_prosperous_hotspots',
       'pct_zcta_hh_clusters', 'pct_zcta_ll_clusters', 'pct_zcta_hl_outliers',
       'pct_zcta_lh_outliers', 'pct_zcta_distressed', 'pct_zcta_prosperous',
       'pct_zcta_at_risk_or_distressed', 'MSA_globalMorans_I', 'MSA_morans_p',
       'MSA_morans_z'],
      dtype='object')

In [ ]:
df1 = pd.read_csv ("/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2013_v4.csv")
df1.columns

Index(['zcta5', 'dci_score', 'quintile', 'pop_zcta_year', 'local_morans_i',
       'local_morans_z', 'local_morans_p', 'quadrant', 'is_hh_cluster',
       'is_ll_cluster', 'is_hl_outlier', 'is_lh_outlier', 'is_distressed',
       'is_prosperous', 'is_at_risk_or_distressed', 'is_distressed_hotspot',
       'is_distressed_coldspot', 'is_prosperous_hotspot',
       'is_prosperous_coldspot'],
      dtype='object')

In [ ]:
df1.shape

(14909, 19)

We have the required columns so moving to the map creation phase

# Map Creation with MSA and Logan's MSA codes

In [ ]:
! pip install mapclassify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 15.0 MB/s eta 0:00:00


In [ ]:
# @title DCI Mapping Suite 2013 (Distress & Clusters)
import os
import glob
import json
import pandas as pd
import geopandas as gpd
import folium
import folium.plugins as plugins
from folium import MacroElement
from shapely.geometry import shape
from jinja2 import Template
import mapclassify

# ---------------------------------------------------------
# 1. SETUP & DATA INGESTION
# ---------------------------------------------------------
print("Step 1: Setting up environment...")

# A. Clone the Repository (if not already present)
if not os.path.exists("us_zips"):
    print("Cloning loganrkerr/us_zips...")
    !git clone https://github.com/loganrkerr/us_zips.git
else:
    print("Repository already exists.")

# B. Load the DCI Data (User's File)
# UPDATE THIS PATH to your actual file location in Colab
csv_path = "/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2013_v4.csv"

# Fallback for testing if file doesn't exist in this session
if not os.path.exists(csv_path):
    print(f"WARNING: File not found at {csv_path}. Please upload it or update the path.")
    # Create dummy data for demonstration if real file is missing
    print("Creating dummy data for demonstration...")
    d = {'zcta5': ['10001', '90210'], 'dci_score': [85.5, 10.2], 'quadrant': [1, 2]}
    df = pd.DataFrame(d)
else:
    print(f"Loading data from {csv_path}...")
    df = pd.read_csv(csv_path)

# Ensure ZCTA is 5-digit string with leading zeros
df['zcta5'] = df['zcta5'].astype(str).str.zfill(5)

# ---------------------------------------------------------
# 2. GEOMETRY CONSTRUCTION (The Heavy Lift)
# ---------------------------------------------------------
print("Step 2: Constructing Geometry from JSON files (This may take a moment)...")

json_files = glob.glob("us_zips/zip_codes_for_the_usa/*.json")
features = []

# Loop through all MSA JSON files
for filepath in json_files:
    with open(filepath, 'r') as f:
        try:
            data = json.load(f)
            # Each file is a FeatureCollection or a list of features
            if 'features' in data:
                for feature in data['features']:
                    # Extract Zip Code from properties (Keys vary, usually 'ZCTA5CE10' or similar)
                    props = feature.get('properties', {})
                    # Find the key that looks like a zip code
                    zip_code = props.get('ZCTA5CE10') or props.get('zip') or props.get('ZIP')

                    if zip_code:
                        features.append({
                            'type': 'Feature',
                            'properties': {'zcta5': str(zip_code)}, # Ensure zcta5 is string
                            'geometry': feature['geometry']
                        })
        except Exception as e:
            print(f"Skipping error in {filepath}: {e}")

# Create GeoDataFrame
print(f"Creating GeoDataFrame from {len(features)} ZCTA geometries...")
gdf = gpd.GeoDataFrame.from_features(features)
gdf.crs = "EPSG:4326" # Assumed WGS84

# Merge with DCI Data
print("Merging Geometry with DCI Data...")
merged_gdf = gdf.merge(df, on='zcta5', how='left')

# Simplify geometry for browser performance (Essential for 30k+ polygons)
# 0.001 degrees is roughly 100m, keeping shapes recognizable but much lighter
print("Simplifying geometries for rendering...")
merged_gdf['geometry'] = merged_gdf['geometry'].simplify(tolerance=0.001, preserve_topology=True)

# Drop rows with no data to keep map clean
# --- MODIFIED SECTION: KEEP ALL GEOMETRY ---

# Instead of dropping NaNs, we fill them with a placeholder
# so the map draws the shape, but we know it's empty.
print(f"Total Geometry Shapes: {len(gdf)}")
print(f"Total Data Rows: {len(df)}")

# Perform the merge (Keep all Geometry, even if no Data match)
merged_gdf = gdf.merge(df, on='zcta5', how='left')

# Do NOT dropna. Instead, fill NaNs for the visual tooltip
merged_gdf['dci_score'] = merged_gdf['dci_score'].fillna(-1) # Use -1 to represent "No Data"
merged_gdf['quadrant'] = merged_gdf['quadrant'].fillna(0)    # Use 0 for Insignificant/No Data

print(f"Mapping {len(merged_gdf)} shapes (some may be gray/empty)...")

print(f"Final dataset has {len(merged_gdf)} features ready for mapping.")


# ---------------------------------------------------------
# 3. MAP 1: DISTRESS CONCENTRATION (Gradient)
# ---------------------------------------------------------
print("Step 3: Building Distress Concentration Map...")

m1 = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="CartoDB positron")

# Add Choropleth
folium.Choropleth(
    geo_data=merged_gdf,
    name="Distress Score",
    data=merged_gdf,
    columns=['zcta5', 'dci_score'],
    key_on='feature.properties.zcta5',
    fill_color='RdYlGn_r',  # Red = High Distress (Bad), Green = Low Distress (Good)
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name='Distress Community Index (DCI Score)',
    smooth_factor=0.5
).add_to(m1)

# Add Hover Tooltip
style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.1, 'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 'color':'#000000', 'fillOpacity': 0.50, 'weight': 0.1}
NIL = folium.features.GeoJson(
    merged_gdf,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['zcta5', 'dci_score'],
        aliases=['Zip Code:', 'DCI Score:'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
)
m1.add_child(NIL)
m1.keep_in_front(NIL)

# Title
title_html = '''
     <h3 align="center" style="font-size:20px"><b>US Distress Concentration (2013)</b></h3>
     '''
m1.get_root().html.add_child(folium.Element(title_html))

print("Saving Map 1...")
m1.save("dci_distress_map_2013.html")


# ---------------------------------------------------------
# 4. MAP 2: LISA CLUSTERS (Categorical)
# ---------------------------------------------------------
print("Step 4: Building LISA Cluster Map...")

m2 = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="CartoDB dark_matter")

# Define Colors for Quadrants
# Adjust these integers (1, 2, 3, 4) based on your actual data values!
# Assumption: 1=HH, 2=LL, 3=HL, 4=LH (Standard PySAL/GeoDa output)
quadrant_colors = {
    1: '#d7191c', # HH - Red (High Distress in High Distress)
    2: '#2b83ba', # LL - Blue (Low Distress in Low Distress)
    3: '#fdae61', # HL - Orange (High Distress outlier in Low area)
    4: '#abd9e9', # LH - Light Blue/Purple (Low Distress outlier in High area)
    0: '#808080'  # Insignificant - Gray
}

def style_fn(feature):
    # Get quadrant value, default to 0 if missing
    q = feature['properties']['quadrant']
    if pd.isna(q): q = 0
    return {
        'fillColor': quadrant_colors.get(int(q), '#808080'),
        'color': 'black',
        'weight': 0.1,
        'fillOpacity': 0.8 if q != 0 else 0.1 # Make insignificant transparent
    }

# Add GeoJson Layer
folium.GeoJson(
    merged_gdf,
    style_function=style_fn,
    name="LISA Clusters",
    tooltip=folium.features.GeoJsonTooltip(
        fields=['zcta5', 'quadrant', 'dci_score'],
        aliases=['Zip Code:', 'Cluster Type:', 'DCI Score:'],
    )
).add_to(m2)

# Custom HTML Legend
legend_html = """
{% macro html(this, kwargs) %}
<style>
.map-legend {
    position: absolute;
    bottom: 50px;
    left: 50px;
    width: 250px;
    z-index: 9999;
    background-color: rgba(255, 255, 255, 0.9);
    padding: 10px;
    border: 1px solid #ccc;
    border-radius: 5px;
    font-family: sans-serif;
    font-size: 12px;
}
.legend-item { margin-bottom: 5px; display: flex; align-items: center; }
.color-box { width: 20px; height: 20px; margin-right: 10px; border: 1px solid #777; }
</style>
<div class="map-legend">
    <h4><b>LISA Cluster Types</b></h4>
    <div class="legend-item"><div class="color-box" style="background:#d7191c;"></div><b>High-High (Hotspot)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#2b83ba;"></div><b>Low-Low (Coldspot)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#fdae61;"></div><b>High-Low (Spatial Outlier)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#abd9e9;"></div><b>Low-High (Spatial Outlier)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#808080; opacity:0.3;"></div>Insignificant</div>
</div>
{% endmacro %}
"""
macro = MacroElement()
macro._template = Template(legend_html)
m2.get_root().add_child(macro)

# Title
title_html_2 = '''
     <h3 align="center" style="color: white; font-size:20px"><b>LISA Clusters (2013)</b></h3>
     '''
m2.get_root().html.add_child(folium.Element(title_html_2))

print("Saving Map 2...")
m2.save("lisa_cluster_map_2013.html")

print("Done! Please check the file browser for 'dci_distress_map_2013.html' and 'lisa_cluster_map_2013.html'")

Step 1: Setting up environment...
Repository already exists.
Loading data from /content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2013_v4.csv...
Step 2: Constructing Geometry from JSON files (This may take a moment)...
Creating GeoDataFrame from 28416 ZCTA geometries...
Merging Geometry with DCI Data...
Simplifying geometries for rendering...
Total Geometry Shapes: 28416
Total Data Rows: 14909
Mapping 28416 shapes (some may be gray/empty)...
Final dataset has 28416 features ready for mapping.
Step 3: Building Distress Concentration Map...
Saving Map 1...
Step 4: Building LISA Cluster Map...
Saving Map 2...
Done! Please check the file browser for 'dci_distress_map_2013.html' and 'lisa_cluster_map_2013.html'


# Only half the map was populate, finding out why this is

In [ ]:
import glob
import json
import pandas as pd

# 1. Load Your CSV Data
print("--- Checking CSV Data ---")
csv_path = "/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2013_v4.csv"
df_debug = pd.read_csv(csv_path)
df_debug['zcta5'] = df_debug['zcta5'].astype(str).str.zfill(5)

# Check counts of Zip codes by first digit (Region proxy)
# 0=Northeast, 3=South, 9=West Coast
print(f"Total rows in CSV: {len(df_debug)}")
print("Zips starting with '0' (Northeast):", len(df_debug[df_debug['zcta5'].str.startswith('0')]))
print("Zips starting with '3' (South):    ", len(df_debug[df_debug['zcta5'].str.startswith('3')]))
print("Zips starting with '9' (West):     ", len(df_debug[df_debug['zcta5'].str.startswith('9')]))

# 2. Check the Repository Files
print("\n--- Checking Geometry Repository ---")
json_files = glob.glob("us_zips/zip_codes_for_the_usa/*.json")
print(f"Total JSON files found: {len(json_files)}")

# Check for specific Western Cities
west_cities = ['San Diego', 'Seattle', 'Los Angeles', 'Portland', 'Denver']
found_cities = []
for city in west_cities:
    match = any(city in f for f in json_files)
    found_cities.append(f"{city}: {'Found' if match else 'MISSING'}")
print("Western MSAs presence:", found_cities)

# 3. Check the Merge Intersection
# Load ONE Western file to see if keys match
san_diego_file = [f for f in json_files if "San Diego" in f]
if san_diego_file:
    with open(san_diego_file[0], 'r') as f:
        data = json.load(f)
        # Get first zip code from this file
        props = data['features'][0]['properties']
        sample_zip = props.get('ZCTA5CE10') or props.get('zip')
        print(f"\nSample Zip from San Diego JSON: '{sample_zip}' (Type: {type(sample_zip)})")

        # Does this specific zip exist in your CSV?
        match = df_debug[df_debug['zcta5'] == str(sample_zip)]
        if not match.empty:
            print("✅ MATCH SUCCESS: This Western zip exists in your CSV.")
        else:
            print("❌ MATCH FAIL: This Western zip exists in geometry but is MISSING from your CSV.")
else:
    print("\n❌ Critical: San Diego JSON file not found in repository.")

--- Checking CSV Data ---
Total rows in CSV: 14909
Zips starting with '0' (Northeast): 1593
Zips starting with '3' (South):     3086
Zips starting with '9' (West):      0

--- Checking Geometry Repository ---
Total JSON files found: 408
Western MSAs presence: ['San Diego: Found', 'Seattle: Found', 'Los Angeles: Found', 'Portland: Found', 'Denver: Found']

Sample Zip from San Diego JSON: 'None' (Type: <class 'NoneType'>)
❌ MATCH FAIL: This Western zip exists in geometry but is MISSING from your CSV.


# It says we dont have the whole West data, lets investigate DCI data that was v2 new


In [ ]:
df2 = pd.read_excel("/content/drive/MyDrive/Saksham_MEFI/DCI/DCI_datasets_longitudinal_zip.xlsx")

In [ ]:
df2.columns

Index(['ZCTA', 'State FIPS', 'County FIPS', 'Total Population',
       '% of Adults w/o a High School Degree', 'Student Adjusted Poverty Rate',
       '% of Prime Age Adults Not in Work', 'Housing Vacancy Rate',
       'Median Household Income Ratio', '5-Year Change in Employment',
       '5-Year Change in Establishments', 'Distress Score',
       'Quintile (5=Distressed)', 'DCI Decile',
       'Rank of Zip w/in County (1=Most Prosperous)',
       'Number of Zips in County',
       'Rank of Zip w/in State (1=Most Prosperous)', 'Number of Zips in State',
       'Employment 2013', 'Employment 2009', 'Establishments 2013',
       'Establishments 2009', 'Median Household Income',
       'Non-Hispanic White % of Population',
       'Hispanic or Latino % of Population',
       'Black or African-American % of Population',
       'American Indian or Alaska Native % of Population',
       'Asian or Pacific Islander % of Population',
       'Other Race or Two or More Races % of Population',
    

In [ ]:
df2.shape

(14909, 33)

# Looking at Logan's TSV file

In [ ]:
df3 = pd.read_csv("/content/us_zips/MSA.tsv", sep='\t')

In [ ]:
df3.columns

Index(['zip_code', 'MSA_number', 'MSA_name'], dtype='object')

In [ ]:
df3.shape

(43487, 3)

#Map for 2017

In [ ]:
# @title DCI Mapping Suite 2017 (Distress & Clusters)
import os
import glob
import json
import pandas as pd
import geopandas as gpd
import folium
import folium.plugins as plugins
from folium import MacroElement
from shapely.geometry import shape
from jinja2 import Template
import mapclassify

# ---------------------------------------------------------
# 1. SETUP & DATA INGESTION
# ---------------------------------------------------------
print("Step 1: Setting up environment...")

# A. Clone the Repository (if not already present)
if not os.path.exists("us_zips"):
    print("Cloning loganrkerr/us_zips...")
    !git clone https://github.com/loganrkerr/us_zips.git
else:
    print("Repository already exists.")

# B. Load the DCI Data (User's File)
# UPDATE THIS PATH to your actual file location in Colab
csv_path = "/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2017_v4.csv"

# Fallback for testing if file doesn't exist in this session
if not os.path.exists(csv_path):
    print(f"WARNING: File not found at {csv_path}. Please upload it or update the path.")
    # Create dummy data for demonstration if real file is missing
    print("Creating dummy data for demonstration...")
    d = {'zcta5': ['10001', '90210'], 'dci_score': [85.5, 10.2], 'quadrant': [1, 2]}
    df = pd.DataFrame(d)
else:
    print(f"Loading data from {csv_path}...")
    df = pd.read_csv(csv_path)

# Ensure ZCTA is 5-digit string with leading zeros
df['zcta5'] = df['zcta5'].astype(str).str.zfill(5)

# ---------------------------------------------------------
# 2. GEOMETRY CONSTRUCTION (The Heavy Lift)
# ---------------------------------------------------------
print("Step 2: Constructing Geometry from JSON files (This may take a moment)...")

json_files = glob.glob("us_zips/zip_codes_for_the_usa/*.json")
features = []

# Loop through all MSA JSON files
for filepath in json_files:
    with open(filepath, 'r') as f:
        try:
            data = json.load(f)
            # Each file is a FeatureCollection or a list of features
            if 'features' in data:
                for feature in data['features']:
                    # Extract Zip Code from properties (Keys vary, usually 'ZCTA5CE10' or similar)
                    props = feature.get('properties', {})
                    # Find the key that looks like a zip code
                    zip_code = props.get('ZCTA5CE10') or props.get('zip') or props.get('ZIP')

                    if zip_code:
                        features.append({
                            'type': 'Feature',
                            'properties': {'zcta5': str(zip_code)}, # Ensure zcta5 is string
                            'geometry': feature['geometry']
                        })
        except Exception as e:
            print(f"Skipping error in {filepath}: {e}")

# Create GeoDataFrame
print(f"Creating GeoDataFrame from {len(features)} ZCTA geometries...")
gdf = gpd.GeoDataFrame.from_features(features)
gdf.crs = "EPSG:4326" # Assumed WGS84

# Merge with DCI Data
print("Merging Geometry with DCI Data...")
merged_gdf = gdf.merge(df, on='zcta5', how='left')

# Simplify geometry for browser performance (Essential for 30k+ polygons)
# 0.001 degrees is roughly 100m, keeping shapes recognizable but much lighter
print("Simplifying geometries for rendering...")
merged_gdf['geometry'] = merged_gdf['geometry'].simplify(tolerance=0.001, preserve_topology=True)

# Drop rows with no data to keep map clean
# --- MODIFIED SECTION: KEEP ALL GEOMETRY ---

# Instead of dropping NaNs, we fill them with a placeholder
# so the map draws the shape, but we know it's empty.
print(f"Total Geometry Shapes: {len(gdf)}")
print(f"Total Data Rows: {len(df)}")

# Perform the merge (Keep all Geometry, even if no Data match)
merged_gdf = gdf.merge(df, on='zcta5', how='left')

# Do NOT dropna. Instead, fill NaNs for the visual tooltip
merged_gdf['dci_score'] = merged_gdf['dci_score'].fillna(-1) # Use -1 to represent "No Data"
merged_gdf['quadrant'] = merged_gdf['quadrant'].fillna(0)    # Use 0 for Insignificant/No Data

print(f"Mapping {len(merged_gdf)} shapes (some may be gray/empty)...")

print(f"Final dataset has {len(merged_gdf)} features ready for mapping.")


# ---------------------------------------------------------
# 3. MAP 1: DISTRESS CONCENTRATION (Gradient)
# ---------------------------------------------------------
print("Step 3: Building Distress Concentration Map...")

m1 = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="CartoDB positron")

# Add Choropleth
folium.Choropleth(
    geo_data=merged_gdf,
    name="Distress Score",
    data=merged_gdf,
    columns=['zcta5', 'dci_score'],
    key_on='feature.properties.zcta5',
    fill_color='RdYlGn_r',  # Red = High Distress (Bad), Green = Low Distress (Good)
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name='Distress Community Index (DCI Score)',
    smooth_factor=0.5
).add_to(m1)

# Add Hover Tooltip
style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.1, 'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 'color':'#000000', 'fillOpacity': 0.50, 'weight': 0.1}
NIL = folium.features.GeoJson(
    merged_gdf,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['zcta5', 'dci_score'],
        aliases=['Zip Code:', 'DCI Score:'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
)
m1.add_child(NIL)
m1.keep_in_front(NIL)

# Title
title_html = '''
     <h3 align="center" style="font-size:20px"><b>US Distress Concentration (2017)</b></h3>
     '''
m1.get_root().html.add_child(folium.Element(title_html))

print("Saving Map 1...")
m1.save("dci_distress_map_2017.html")


# ---------------------------------------------------------
# 4. MAP 2: LISA CLUSTERS (Categorical)
# ---------------------------------------------------------
print("Step 4: Building LISA Cluster Map...")

m2 = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="CartoDB dark_matter")

# Define Colors for Quadrants
# Adjust these integers (1, 2, 3, 4) based on your actual data values!
# Assumption: 1=HH, 2=LL, 3=HL, 4=LH (Standard PySAL/GeoDa output)
quadrant_colors = {
    1: '#d7191c', # HH - Red (High Distress in High Distress)
    2: '#2b83ba', # LL - Blue (Low Distress in Low Distress)
    3: '#fdae61', # HL - Orange (High Distress outlier in Low area)
    4: '#abd9e9', # LH - Light Blue/Purple (Low Distress outlier in High area)
    0: '#808080'  # Insignificant - Gray
}

def style_fn(feature):
    # Get quadrant value, default to 0 if missing
    q = feature['properties']['quadrant']
    if pd.isna(q): q = 0
    return {
        'fillColor': quadrant_colors.get(int(q), '#808080'),
        'color': 'black',
        'weight': 0.1,
        'fillOpacity': 0.8 if q != 0 else 0.1 # Make insignificant transparent
    }

# Add GeoJson Layer
folium.GeoJson(
    merged_gdf,
    style_function=style_fn,
    name="LISA Clusters",
    tooltip=folium.features.GeoJsonTooltip(
        fields=['zcta5', 'quadrant', 'dci_score'],
        aliases=['Zip Code:', 'Cluster Type:', 'DCI Score:'],
    )
).add_to(m2)

# Custom HTML Legend
legend_html = """
{% macro html(this, kwargs) %}
<style>
.map-legend {
    position: absolute;
    bottom: 50px;
    left: 50px;
    width: 250px;
    z-index: 9999;
    background-color: rgba(255, 255, 255, 0.9);
    padding: 10px;
    border: 1px solid #ccc;
    border-radius: 5px;
    font-family: sans-serif;
    font-size: 12px;
}
.legend-item { margin-bottom: 5px; display: flex; align-items: center; }
.color-box { width: 20px; height: 20px; margin-right: 10px; border: 1px solid #777; }
</style>
<div class="map-legend">
    <h4><b>LISA Cluster Types</b></h4>
    <div class="legend-item"><div class="color-box" style="background:#d7191c;"></div><b>High-High (Hotspot)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#2b83ba;"></div><b>Low-Low (Coldspot)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#fdae61;"></div><b>High-Low (Spatial Outlier)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#abd9e9;"></div><b>Low-High (Spatial Outlier)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#808080; opacity:0.3;"></div>Insignificant</div>
</div>
{% endmacro %}
"""
macro = MacroElement()
macro._template = Template(legend_html)
m2.get_root().add_child(macro)

# Title
title_html_2 = '''
     <h3 align="center" style="color: white; font-size:20px"><b>LISA Clusters (2017)</b></h3>
     '''
m2.get_root().html.add_child(folium.Element(title_html_2))

print("Saving Map 2...")
m2.save("lisa_cluster_map_2017.html")

print("Done! Please check the file browser for 'dci_distress_map_2017.html' and 'lisa_cluster_map_2017.html'")

Step 1: Setting up environment...
Repository already exists.
Loading data from /content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2017_v4.csv...
Step 2: Constructing Geometry from JSON files (This may take a moment)...
Creating GeoDataFrame from 28416 ZCTA geometries...
Merging Geometry with DCI Data...
Simplifying geometries for rendering...
Total Geometry Shapes: 28416
Total Data Rows: 14832
Mapping 28416 shapes (some may be gray/empty)...
Final dataset has 28416 features ready for mapping.
Step 3: Building Distress Concentration Map...
Saving Map 1...
Step 4: Building LISA Cluster Map...
Saving Map 2...
Done! Please check the file browser for 'dci_distress_map_2017.html' and 'lisa_cluster_map_2017.html'


In [ ]:
# @title DCI Mapping Suite 2022 (Distress & Clusters)
import os
import glob
import json
import pandas as pd
import geopandas as gpd
import folium
import folium.plugins as plugins
from folium import MacroElement
from shapely.geometry import shape
from jinja2 import Template
import mapclassify

# ---------------------------------------------------------
# 1. SETUP & DATA INGESTION
# ---------------------------------------------------------
print("Step 1: Setting up environment...")

# A. Clone the Repository (if not already present)
if not os.path.exists("us_zips"):
    print("Cloning loganrkerr/us_zips...")
    !git clone https://github.com/loganrkerr/us_zips.git
else:
    print("Repository already exists.")

# B. Load the DCI Data (User's File)
# UPDATE THIS PATH to your actual file location in Colab
csv_path = "/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2022_v4.csv"

# Fallback for testing if file doesn't exist in this session
if not os.path.exists(csv_path):
    print(f"WARNING: File not found at {csv_path}. Please upload it or update the path.")
    # Create dummy data for demonstration if real file is missing
    print("Creating dummy data for demonstration...")
    d = {'zcta5': ['10001', '90210'], 'dci_score': [85.5, 10.2], 'quadrant': [1, 2]}
    df = pd.DataFrame(d)
else:
    print(f"Loading data from {csv_path}...")
    df = pd.read_csv(csv_path)

# Ensure ZCTA is 5-digit string with leading zeros
df['zcta5'] = df['zcta5'].astype(str).str.zfill(5)

# ---------------------------------------------------------
# 2. GEOMETRY CONSTRUCTION (The Heavy Lift)
# ---------------------------------------------------------
print("Step 2: Constructing Geometry from JSON files (This may take a moment)...")

json_files = glob.glob("us_zips/zip_codes_for_the_usa/*.json")
features = []

# Loop through all MSA JSON files
for filepath in json_files:
    with open(filepath, 'r') as f:
        try:
            data = json.load(f)
            # Each file is a FeatureCollection or a list of features
            if 'features' in data:
                for feature in data['features']:
                    # Extract Zip Code from properties (Keys vary, usually 'ZCTA5CE10' or similar)
                    props = feature.get('properties', {})
                    # Find the key that looks like a zip code
                    zip_code = props.get('ZCTA5CE10') or props.get('zip') or props.get('ZIP')

                    if zip_code:
                        features.append({
                            'type': 'Feature',
                            'properties': {'zcta5': str(zip_code)}, # Ensure zcta5 is string
                            'geometry': feature['geometry']
                        })
        except Exception as e:
            print(f"Skipping error in {filepath}: {e}")

# Create GeoDataFrame
print(f"Creating GeoDataFrame from {len(features)} ZCTA geometries...")
gdf = gpd.GeoDataFrame.from_features(features)
gdf.crs = "EPSG:4326" # Assumed WGS84

# Merge with DCI Data
print("Merging Geometry with DCI Data...")
merged_gdf = gdf.merge(df, on='zcta5', how='left')

# Simplify geometry for browser performance (Essential for 30k+ polygons)
# 0.001 degrees is roughly 100m, keeping shapes recognizable but much lighter
print("Simplifying geometries for rendering...")
merged_gdf['geometry'] = merged_gdf['geometry'].simplify(tolerance=0.001, preserve_topology=True)

# Drop rows with no data to keep map clean
# --- MODIFIED SECTION: KEEP ALL GEOMETRY ---

# Instead of dropping NaNs, we fill them with a placeholder
# so the map draws the shape, but we know it's empty.
print(f"Total Geometry Shapes: {len(gdf)}")
print(f"Total Data Rows: {len(df)}")

# Perform the merge (Keep all Geometry, even if no Data match)
merged_gdf = gdf.merge(df, on='zcta5', how='left')

# Do NOT dropna. Instead, fill NaNs for the visual tooltip
merged_gdf['dci_score'] = merged_gdf['dci_score'].fillna(-1) # Use -1 to represent "No Data"
merged_gdf['quadrant'] = merged_gdf['quadrant'].fillna(0)    # Use 0 for Insignificant/No Data

print(f"Mapping {len(merged_gdf)} shapes (some may be gray/empty)...")

print(f"Final dataset has {len(merged_gdf)} features ready for mapping.")


# ---------------------------------------------------------
# 3. MAP 1: DISTRESS CONCENTRATION (Gradient)
# ---------------------------------------------------------
print("Step 3: Building Distress Concentration Map...")

m1 = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="CartoDB positron")

# Add Choropleth
folium.Choropleth(
    geo_data=merged_gdf,
    name="Distress Score",
    data=merged_gdf,
    columns=['zcta5', 'dci_score'],
    key_on='feature.properties.zcta5',
    fill_color='RdYlGn_r',  # Red = High Distress (Bad), Green = Low Distress (Good)
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name='Distress Community Index (DCI Score)',
    smooth_factor=0.5
).add_to(m1)

# Add Hover Tooltip
style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.1, 'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 'color':'#000000', 'fillOpacity': 0.50, 'weight': 0.1}
NIL = folium.features.GeoJson(
    merged_gdf,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['zcta5', 'dci_score'],
        aliases=['Zip Code:', 'DCI Score:'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
)
m1.add_child(NIL)
m1.keep_in_front(NIL)

# Title
title_html = '''
     <h3 align="center" style="font-size:20px"><b>US Distress Concentration (2022)</b></h3>
     '''
m1.get_root().html.add_child(folium.Element(title_html))

print("Saving Map 1...")
m1.save("dci_distress_map_2022.html")


# ---------------------------------------------------------
# 4. MAP 2: LISA CLUSTERS (Categorical)
# ---------------------------------------------------------
print("Step 4: Building LISA Cluster Map...")

m2 = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="CartoDB dark_matter")

# Define Colors for Quadrants
# Adjust these integers (1, 2, 3, 4) based on your actual data values!
# Assumption: 1=HH, 2=LL, 3=HL, 4=LH (Standard PySAL/GeoDa output)
quadrant_colors = {
    1: '#d7191c', # HH - Red (High Distress in High Distress)
    2: '#2b83ba', # LL - Blue (Low Distress in Low Distress)
    3: '#fdae61', # HL - Orange (High Distress outlier in Low area)
    4: '#abd9e9', # LH - Light Blue/Purple (Low Distress outlier in High area)
    0: '#808080'  # Insignificant - Gray
}

def style_fn(feature):
    # Get quadrant value, default to 0 if missing
    q = feature['properties']['quadrant']
    if pd.isna(q): q = 0
    return {
        'fillColor': quadrant_colors.get(int(q), '#808080'),
        'color': 'black',
        'weight': 0.1,
        'fillOpacity': 0.8 if q != 0 else 0.1 # Make insignificant transparent
    }

# Add GeoJson Layer
folium.GeoJson(
    merged_gdf,
    style_function=style_fn,
    name="LISA Clusters",
    tooltip=folium.features.GeoJsonTooltip(
        fields=['zcta5', 'quadrant', 'dci_score'],
        aliases=['Zip Code:', 'Cluster Type:', 'DCI Score:'],
    )
).add_to(m2)

# Custom HTML Legend
legend_html = """
{% macro html(this, kwargs) %}
<style>
.map-legend {
    position: absolute;
    bottom: 50px;
    left: 50px;
    width: 250px;
    z-index: 9999;
    background-color: rgba(255, 255, 255, 0.9);
    padding: 10px;
    border: 1px solid #ccc;
    border-radius: 5px;
    font-family: sans-serif;
    font-size: 12px;
}
.legend-item { margin-bottom: 5px; display: flex; align-items: center; }
.color-box { width: 20px; height: 20px; margin-right: 10px; border: 1px solid #777; }
</style>
<div class="map-legend">
    <h4><b>LISA Cluster Types</b></h4>
    <div class="legend-item"><div class="color-box" style="background:#d7191c;"></div><b>High-High (Hotspot)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#2b83ba;"></div><b>Low-Low (Coldspot)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#fdae61;"></div><b>High-Low (Spatial Outlier)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#abd9e9;"></div><b>Low-High (Spatial Outlier)</b></div>
    <div class="legend-item"><div class="color-box" style="background:#808080; opacity:0.3;"></div>Insignificant</div>
</div>
{% endmacro %}
"""
macro = MacroElement()
macro._template = Template(legend_html)
m2.get_root().add_child(macro)

# Title
title_html_2 = '''
     <h3 align="center" style="color: white; font-size:20px"><b>LISA Clusters (2022)</b></h3>
     '''
m2.get_root().html.add_child(folium.Element(title_html_2))

print("Saving Map 2...")
m2.save("lisa_cluster_map_2022.html")

print("Done! Please check the file browser for 'dci_distress_map_2022.html' and 'lisa_cluster_map_2022.html'")

Step 1: Setting up environment...
Repository already exists.
Loading data from /content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2022_v4.csv...
Step 2: Constructing Geometry from JSON files (This may take a moment)...
Creating GeoDataFrame from 28416 ZCTA geometries...
Merging Geometry with DCI Data...
Simplifying geometries for rendering...
Total Geometry Shapes: 28416
Total Data Rows: 14745
Mapping 28416 shapes (some may be gray/empty)...
Final dataset has 28416 features ready for mapping.
Step 3: Building Distress Concentration Map...
Saving Map 1...
Step 4: Building LISA Cluster Map...
Saving Map 2...
Done! Please check the file browser for 'dci_distress_map_2022.html' and 'lisa_cluster_map_2022.html'


# Temporal Lag Maps

In [ ]:
# @title MEFI Temporal Mapping Suite (2013 -> 2017 -> 2022)
import os
import glob
import json
import pandas as pd
import geopandas as gpd
import folium
from folium import MacroElement
from shapely.geometry import shape

# ---------------------------------------------------------
# 1. SETUP & GLOBAL CONFIGURATION
# ---------------------------------------------------------
print("Step 1: Setting up environment...")

# A. Clone Repository (Geometry Source)
if not os.path.exists("us_zips"):
    print("Cloning loganrkerr/us_zips for geometry...")
    !git clone https://github.com/loganrkerr/us_zips.git
else:
    print("Geometry repository already active.")

# B. Define File Paths for Each Year
# UPDATE THESE PATHS to match your Google Drive location exactly
data_files = {
    2013: "/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2013_v4.csv",
    2017: "/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2017_v4.csv",
    2022: "/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2022_v4.csv"
}

# C. Define Color Schemes
quadrant_colors = {
    1: '#d7191c', # HH (Red)
    2: '#2b83ba', # LL (Blue)
    3: '#fdae61', # HL (Orange)
    4: '#abd9e9', # LH (Light Blue/Purple)
    0: '#cccccc'  # Insignificant/No Data (Gray)
}

# ---------------------------------------------------------
# 2. GEOMETRY ENGINE (Build Once, Use Many Times)
# ---------------------------------------------------------
print("Step 2: Constructing Nationwide ZCTA Geometry...")
# We build the master geometry file FIRST so we don't have to re-parse JSONs 3 times.

json_files = glob.glob("us_zips/zip_codes_for_the_usa/*.json")
features = []

for filepath in json_files:
    with open(filepath, 'r') as f:
        try:
            data = json.load(f)
            if 'features' in data:
                for feature in data['features']:
                    props = feature.get('properties', {})
                    # Try common keys for zip codes
                    zip_code = props.get('ZCTA5CE10') or props.get('zip') or props.get('ZIP')

                    if zip_code:
                        features.append({
                            'type': 'Feature',
                            'properties': {'zcta5': str(zip_code).zfill(5)}, # Normalizing here
                            'geometry': feature['geometry']
                        })
        except Exception as e:
            pass # Skip corrupt files

print(f"  - Parsed {len(features)} ZCTA shapes.")
master_gdf = gpd.GeoDataFrame.from_features(features)
master_gdf.crs = "EPSG:4326"

# Simplify NOW to save memory for the rest of the script
print("  - Simplifying geometry (tolerance=0.001)...")
master_gdf['geometry'] = master_gdf['geometry'].simplify(tolerance=0.001, preserve_topology=True)

# ---------------------------------------------------------
# 3. TEMPORAL MAPPING LOOP
# ---------------------------------------------------------
print("\nStep 3: Generating Maps for All Years...")

for year, filepath in data_files.items():
    print(f"\n--- Processing Year: {year} ---")

    # A. Load Data
    if not os.path.exists(filepath):
        print(f"  [WARNING] File not found for {year}: {filepath}. Skipping.")
        continue

    df = pd.read_csv(filepath)
    df['zcta5'] = df['zcta5'].astype(str).str.zfill(5)

    # B. Merge with Master Geometry
    # We use 'left' merge on the Geometry.
    # If data is missing (Western US), the columns 'dci_score' etc. will be NaN.
    merged_gdf = master_gdf.merge(df, on='zcta5', how='left')

    # C. Handle Missing Data (The Fix for Western US)
    # Fill NaNs with placeholders so they appear on map as Gray instead of invisible
    merged_gdf['dci_score'] = merged_gdf['dci_score'].fillna(-1)
    merged_gdf['quadrant'] = merged_gdf['quadrant'].fillna(0)

    print(f"  - Merged data. Total shapes: {len(merged_gdf)}")

    # --- MAP 1: DISTRESS GRADIENT ---
    print(f"  - Building Distress Map ({year})...")
    m1 = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="CartoDB positron")

    # Add Gradient Layer
    folium.Choropleth(
        geo_data=merged_gdf,
        name="Distress Score",
        data=merged_gdf,
        columns=['zcta5', 'dci_score'],
        key_on='feature.properties.zcta5',
        fill_color='RdYlGn_r',
        bins=8, # Auto-binning
        nan_fill_color="#cccccc", # Explicit gray for missing data
        fill_opacity=0.7,
        line_opacity=0.1,
        legend_name=f'MEFI Distress Score ({year})'
    ).add_to(m1)

    # Add Title
    title_html = f'<h3 align="center" style="font-size:20px"><b>MEFI Distress Concentration ({year})</b></h3>'
    m1.get_root().html.add_child(folium.Element(title_html))

    m1.save(f"mefi_distress_map_{year}.html")

    # --- MAP 2: LISA CLUSTERS ---
    print(f"  - Building Cluster Map ({year})...")
    m2 = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="CartoDB dark_matter")

    # Style Function for Categories
    def style_fn(feature):
        q = feature['properties']['quadrant']
        # If NaN or 0, return Gray
        if pd.isna(q) or q == 0:
            col = '#cccccc'
            opac = 0.1 # Transparent if no data
        else:
            col = quadrant_colors.get(int(q), '#cccccc')
            opac = 0.8

        return {'fillColor': col, 'color': 'black', 'weight': 0.1, 'fillOpacity': opac}

    # Add GeoJson Layer
    folium.GeoJson(
        merged_gdf,
        style_function=style_fn,
        name="LISA Clusters",
        tooltip=folium.features.GeoJsonTooltip(
            fields=['zcta5', 'quadrant', 'dci_score'],
            aliases=['Zip Code:', 'Cluster Type:', 'Score:'],
        )
    ).add_to(m2)

    # Custom Legend (Same as before)
    legend_html = """
    {% macro html(this, kwargs) %}
    <style>
    .map-legend { position: absolute; bottom: 50px; left: 50px; width: 250px; z-index: 9999; background-color: rgba(255, 255, 255, 0.9); padding: 10px; border: 1px solid #ccc; border-radius: 5px; font-family: sans-serif; font-size: 12px; } boat
    .legend-item { margin-bottom: 5px; display: flex; align-items: center; } boat
    .color-box { width: 20px; height: 20px; margin-right: 10px; border: 1px solid #777; } boat
    </style> boat
    <div class="map-legend"> boat
        <h4><b>LISA Cluster Types</b></h4> boat
        <div class="legend-item"><div class="color-box" style="background:#d7191c;"></div><b>High-High (Hotspot)</b></div> boat
        <div class="legend-item"><div class="color-box" style="background:#2b83ba;"></div><b>Low-Low (Coldspot)</b></div> boat
        <div class="legend-item"><div class="color-box" style="background:#fdae61;"></div><b>High-Low (Outlier)</b></div> boat
        <div class="legend-item"><div class="color-box" style="background:#abd9e9;"></div><b>Low-High (Outlier)</b></div> boat
        <div class="legend-item"><div class="color-box" style="background:#cccccc; opacity:0.3;"></div>No Data / Insignificant</div> boat
    </div> boat
    {% endmacro %} boat
    """
    macro = MacroElement()
    macro._template = Template(legend_html)
    m2.get_root().add_child(macro)

    # Title
    title_html_2 = f'<h3 align="center" style="color: white; font-size:20px"><b>MEFI LISA Clusters ({year})</b></h3>'
    m2.get_root().html.add_child(folium.Element(title_html_2))

    m2.save(f"mefi_cluster_map_{year}.html")

print("\nSuccess! All 6 maps have been generated.")

Step 1: Setting up environment...
Geometry repository already active.
Step 2: Constructing Nationwide ZCTA Geometry...
  - Parsed 28416 ZCTA shapes.
  - Simplifying geometry (tolerance=0.001)...

Step 3: Generating Maps for All Years...

--- Processing Year: 2013 ---
  - Merged data. Total shapes: 28416
  - Building Distress Map (2013)...
  - Building Cluster Map (2013)...

--- Processing Year: 2017 ---
  - Merged data. Total shapes: 28416
  - Building Distress Map (2017)...
  - Building Cluster Map (2017)...

--- Processing Year: 2022 ---
  - Merged data. Total shapes: 28416
  - Building Distress Map (2022)...
  - Building Cluster Map (2022)...

Success! All 6 maps have been generated.


# final maps generated

In [ ]:
# @title DCI Mapping Suite (2013, 2017, 2022)
import os
import glob
import json
import pandas as pd
import geopandas as gpd
import folium
from folium import MacroElement
from jinja2 import Template
from google.colab import files
import shutil

# ---------------------------------------------------------
# 1. SETUP & GEOMETRY CONSTRUCTION (Run Once)
# ---------------------------------------------------------
print("Step 1: Setting up environment and geometry...")

# A. Clone the Repository
if not os.path.exists("us_zips"):
    print("Cloning loganrkerr/us_zips...")
    !git clone https://github.com/loganrkerr/us_zips.git
else:
    print("Repository already exists.")

# B. Construct Geometry (Only doing this once to save time)
if 'base_gdf' not in locals(): # Check if we already loaded it to avoid reloading
    print("Constructing Base Geometry from JSON files...")
    json_files = glob.glob("us_zips/zip_codes_for_the_usa/*.json")
    features = []

    for filepath in json_files:
        with open(filepath, 'r') as f:
            try:
                data = json.load(f)
                if 'features' in data:
                    for feature in data['features']:
                        props = feature.get('properties', {})
                        zip_code = props.get('ZCTA5CE10') or props.get('zip') or props.get('ZIP')
                        if zip_code:
                            features.append({
                                'type': 'Feature',
                                'properties': {'zcta5': str(zip_code)},
                                'geometry': feature['geometry']
                            })
            except Exception as e:
                pass # Skip errors silently for speed

    base_gdf = gpd.GeoDataFrame.from_features(features)
    base_gdf.crs = "EPSG:4326"

    # Simplify immediately
    print("Simplifying geometries...")
    base_gdf['geometry'] = base_gdf['geometry'].simplify(tolerance=0.001, preserve_topology=True)

# ---------------------------------------------------------
# 2. PROCESSING LOOP (2013, 2017, 2022)
# ---------------------------------------------------------

data_files = {
    2013: "/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2013_v4.csv",
    2017: "/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2017_v4.csv",
    2022: "/content/drive/MyDrive/Saksham_MEFI/DCI/outputs_final/zcta_spatial_2022_v4.csv"
}

output_dir = "/content/mefi_maps_output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for year, csv_path in data_files.items():
    print(f"--- Processing Year: {year} ---")

    # Load Data
    if not os.path.exists(csv_path):
        print(f"WARNING: File not found for {year} at {csv_path}. Skipping.")
        continue

    df = pd.read_csv(csv_path)
    df['zcta5'] = df['zcta5'].astype(str).str.zfill(5)

    # Merge with Base Geometry
    # We use a fresh copy of base_gdf so we don't corrupt the original
    merged_gdf = base_gdf.copy().merge(df, on='zcta5', how='left')

    # Fill NaNs for Tooltip display, but keep raw for logic
    merged_gdf['display_dci'] = merged_gdf['dci_score'].fillna("No Data")
    merged_gdf['display_quad'] = merged_gdf['quadrant'].fillna("No Data")

    # ---------------------------------------------------------
    # MAP 1: DISTRESS (Green Background for Empty)
    # ---------------------------------------------------------
    m1 = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="CartoDB positron")

    # Color Logic:
    # We use 'nan_fill_color' to handle the "remaining spaces" (empty data)
    folium.Choropleth(
        geo_data=merged_gdf,
        name="Distress Score",
        data=merged_gdf,
        columns=['zcta5', 'dci_score'],
        key_on='feature.properties.zcta5',
        fill_color='RdYlGn_r',
        fill_opacity=0.7,
        line_opacity=0.1,
        legend_name=f'DCI Score ({year})',
        nan_fill_color='green',   # <--- USER REQUEST: Empty spaces are Green
        nan_fill_opacity=0.4      # Slightly transparent so we can tell it's background
    ).add_to(m1)

    # Title
    title_html = f'''
         <h3 align="center" style="font-size:20px"><b>US Distress Concentration ({year})</b></h3>
         '''
    m1.get_root().html.add_child(folium.Element(title_html))

    # Save
    m1_filename = f"{output_dir}/dci_distress_map_{year}.html"
    m1.save(m1_filename)
    print(f"Saved {m1_filename}")

    # ---------------------------------------------------------
    # MAP 2: LISA CLUSTERS (Blue Background for Empty)
    # ---------------------------------------------------------
    m2 = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="CartoDB dark_matter")

    quadrant_colors = {
        1: '#d7191c', # HH
        2: '#2b83ba', # LL
        3: '#fdae61', # HL
        4: '#abd9e9', # LH
        0: '#808080'  # Insignificant
    }

    def style_fn(feature):
        # Check if data exists for this feature
        # In the merge, if no match found, 'quadrant' is NaN (float)
        q = feature['properties'].get('quadrant')

        if pd.isna(q):
            # <--- USER REQUEST: Empty spaces (No Data) are Blue
            return {'fillColor': 'blue', 'color': 'black', 'weight': 0.1, 'fillOpacity': 0.3}

        # If data exists but is insignificant (0), usually gray
        return {
            'fillColor': quadrant_colors.get(int(q), '#808080'),
            'color': 'black',
            'weight': 0.1,
            'fillOpacity': 0.8 if int(q) != 0 else 0.1
        }

    folium.GeoJson(
        merged_gdf,
        style_function=style_fn,
        name="LISA Clusters",
        tooltip=folium.features.GeoJsonTooltip(
            fields=['zcta5', 'display_quad', 'display_dci'],
            aliases=['Zip:', 'Cluster:', 'DCI:'],
        )
    ).add_to(m2)

    # Title
    title_html_2 = f'''
         <h3 align="center" style="color: white; font-size:20px"><b>LISA Clusters ({year})</b></h3>
         '''
    m2.get_root().html.add_child(folium.Element(title_html_2))

    # Legend (Updated with Blue for No Data)
    legend_html = """
    {% macro html(this, kwargs) %}
    <style>
    .map-legend {
        position: absolute; bottom: 50px; left: 50px; width: 250px; z-index: 9999;
        background-color: rgba(255, 255, 255, 0.9); padding: 10px; border-radius: 5px;
        font-family: sans-serif; font-size: 12px;
    }
    .legend-item { margin-bottom: 5px; display: flex; align-items: center; }
    .color-box { width: 20px; height: 20px; margin-right: 10px; border: 1px solid #777; }
    </style>
    <div class="map-legend">
        <h4><b>LISA Cluster Types</b></h4>
        <div class="legend-item"><div class="color-box" style="background:#d7191c;"></div><b>High-High</b></div>
        <div class="legend-item"><div class="color-box" style="background:#2b83ba;"></div><b>Low-Low</b></div>
        <div class="legend-item"><div class="color-box" style="background:#fdae61;"></div><b>High-Low</b></div>
        <div class="legend-item"><div class="color-box" style="background:#abd9e9;"></div><b>Low-High</b></div>
        <div class="legend-item"><div class="color-box" style="background:#808080; opacity:0.3;"></div>Insignificant</div>
        <div class="legend-item"><div class="color-box" style="background:blue; opacity:0.3;"></div>No Data</div>
    </div>
    {% endmacro %}
    """
    macro = MacroElement()
    macro._template = Template(legend_html)
    m2.get_root().add_child(macro)

    m2_filename = f"{output_dir}/lisa_cluster_map_{year}.html"
    m2.save(m2_filename)
    print(f"Saved {m2_filename}")

# ---------------------------------------------------------
# 3. DOWNLOAD TO LOCAL
# ---------------------------------------------------------
print("Zipping files for download...")
shutil.make_archive("/content/mefi_maps", 'zip', output_dir)

print(f"Downloading zip file. Please extract this to /Users/tarive/mefi_maps/public")
files.download("/content/mefi_maps.zip")

Step 1: Setting up environment and geometry...
Repository already exists.
Constructing Base Geometry from JSON files...
Simplifying geometries...
--- Processing Year: 2013 ---
Saved /content/mefi_maps_output/dci_distress_map_2013.html
Saved /content/mefi_maps_output/lisa_cluster_map_2013.html
--- Processing Year: 2017 ---
Saved /content/mefi_maps_output/dci_distress_map_2017.html
Saved /content/mefi_maps_output/lisa_cluster_map_2017.html
--- Processing Year: 2022 ---
Saved /content/mefi_maps_output/dci_distress_map_2022.html
Saved /content/mefi_maps_output/lisa_cluster_map_2022.html
Zipping files for download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>